### IMPORT SNOWFLAKE SESSION, FUNCTIONS, DATA TYPES AND OTHER PYTHON LIBRARIES

In [1]:
# Import required libraries
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import avg, sum, col,lit
from snowflake.snowpark.functions import udf, sproc, col
from snowflake.snowpark.types import IntegerType, FloatType, LongType, DoubleType, DecimalType,StringType, BooleanType, Variant
from snowflake.snowpark.types import PandasSeries, PandasDataFrame
from snowflake.snowpark import functions as fn

import pandas as pd
import numpy as np
import json

from snowflake.snowpark import version
print (f"snowflake snowpark version is: {version.VERSION}")

snowflake snowpark version is: (0, 10, 0)


### SET UP SNOWFLAKE CONNECTION USING THE CREDENTIALS & PRINT CURRENT DB, SCHEMA AND WAREHOUSE

In [2]:
snowflake_connection_cfg = open('cred.json')
snowflake_connection_cfg = snowflake_connection_cfg.read()
snowflake_connection_cfg = json.loads(snowflake_connection_cfg)

# Creating Snowpark Session
spe_session = Session.builder.configs(snowflake_connection_cfg).create()
print('Current Database:', spe_session.get_current_database())
print('Current Schema:', spe_session.get_current_schema())
print('Current Warehouse:', spe_session.get_current_warehouse())

Current Database: "BANK1_CRM_DB"
Current Schema: "PUBLIC"
Current Warehouse: "APP_WH"


### REMOVE ALL IMPORTS AND PACKAGES IN CURRENT SESSION AND ADD THE NEEDED PACKAGES INCLUDING SNOWFLAKE-SNOWFPARK FOR PYTHON

In [3]:
spe_session.clear_imports()
spe_session.clear_packages()
spe_session.add_packages("snowflake-snowpark-python")
spe_session.add_packages("scikit-learn","pandas","numpy","joblib","cachetools")

### CREATE A NEW INTERNAL SNOWFLAKE STAGE LOCATION TO SAVE THE MODEL FILE LATER

In [4]:
spe_session.sql("CREATE OR REPLACE STAGE knadadur_models").collect()

[Row(status='Stage area KNADADUR_MODELS successfully created.')]

In [5]:
spe_session.sql("list @knadadur_models").collect()

[]

### CREATE A NEW INTERNAL SNOWFLAKE STAGE LOCATION TO SAVE THE INPUT DATA FILE

In [ ]:
spe_session.sql("CREATE STAGE IF NOT EXISTS knadadur_data").collect()

### MAKE SURE YOU ARE LOCATE LOCAL DATA FILE AND LOAD IT TO INTERNAL STAGE

In [6]:
folder_path = "/Users/knadadur/Documents/Snowparkusecases/SoftwareProjectEstimation/"
input_file = "softwareprojectsdata.csv"
input_data = folder_path+input_file
input_data

'/Users/knadadur/Documents/Snowparkusecases/SoftwareProjectEstimation/softwareprojectsdata.csv'

In [ ]:
spe_session.sql("put file:///Users/knadadur/Documents/Snowparkusecases/SoftwareProjectEstimation/softwareprojectsdata.csv @knadadur_data").collect();

In [7]:
pd.read_csv(input_file)

,id,Project,TeamExp,ManagerExp,YearEnd,Length,Effort,Transactions,Entities,PointsNonAdjust,Adjustment,PointsAjust,Language
0,1,1,1,4,85,12,5152,253,52,305,34,302,1
1,2,2,0,0,86,4,5635,197,124,321,33,315,1
2,3,3,4,4,85,1,805,40,60,100,18,83,1
3,4,4,0,0,86,5,3829,200,119,319,30,303,1
4,5,5,0,0,86,4,2149,140,94,234,24,208,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,77,77,4,4,85,12,1400,229,169,398,39,414,3
77,78,78,4,3,83,12,2800,227,73,300,34,297,1
78,79,79,4,4,82,24,9520,395,193,588,40,617,1
79,80,80,4,3,86,12,5880,469,176,645,43,697,3


### CHECK IF THE FILE HAS BEEN UPLOADED TO INTERNAL STAGE

In [8]:
spe_session.sql("list @knadadur_data").collect()

[Row(name='knadadur_data/EcommerceCustomers.csv.gz', size=33792, md5='57705477516e68ab8b3c8401a73deb8c', last_modified='Thu, 3 Nov 2022 17:13:04 GMT'),
 Row(name='knadadur_data/softwareprojectsdata.csv.gz', size=1664, md5='54d6a4fb52738250c895b3e18b47df36', last_modified='Mon, 7 Nov 2022 19:33:43 GMT')]

### CREATE A TABLE IN DATABASE AS WE KNOW THE STRUCTURE

In [9]:
spe_session.sql("CREATE OR REPLACE table SOFTWARE_PROJECTS (ID VARCHAR,PROJECT VARCHAR,TEAMEXP NUMBER,MANAGEREXP NUMBER,YEAREND NUMBER,LENGTH NUMBER,EFFORT NUMBER,TRANSACTIONS NUMBER,ENTITIES NUMBER,POINTNONADJUST NUMBER,ADJUSTMENT NUMBER,POINTSADJUST NUMBER,LANGUAGE NUMBER)").collect()

[Row(status='Table SOFTWARE_PROJECTS successfully created.')]

### NOW LET'S LOAD THE DATA FROM INTERNAL STAGE TO THE software_projects DATABASE TABLE IN SNOWFLAKE

In [10]:
spe_session.sql("copy into SOFTWARE_PROJECTS from @knadadur_data/softwareprojectsdata.csv.gz ON_ERROR = CONTINUE").collect()

[Row(file='knadadur_data/softwareprojectsdata.csv.gz', status='PARTIALLY_LOADED', rows_parsed=82, rows_loaded=81, error_limit=82, errors_seen=1, first_error="Numeric value 'TeamExp' is not recognized", first_error_line=1, first_error_character=13, first_error_column_name='"SOFTWARE_PROJECTS"["TEAMEXP":3]')]

### CHECK THE DATA AND CHECK SOME STATS

In [11]:
snowpark_df = spe_session.table('SOFTWARE_PROJECTS')

# Describing the data
print('Rows in dataset:', f"{snowpark_df.count():,}")
print('Data before imputation:')
display(snowpark_df.describe().sort('SUMMARY').toPandas())

Rows in dataset: 81
Data before imputation:


,SUMMARY,ID,PROJECT,TEAMEXP,MANAGEREXP,YEAREND,LENGTH,EFFORT,TRANSACTIONS,ENTITIES,POINTNONADJUST,ADJUSTMENT,POINTSADJUST,LANGUAGE
0,count,81,81,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
1,max,9,9,4.000000,7.000000,88.000000,39.000000,23940.000000,886.000000,387.000000,1127.000000,52.000000,1116.000000,3.000000
2,mean,None,None,2.185185,2.530864,85.740741,11.666667,5046.308642,182.123457,122.333333,304.456790,27.629630,289.234568,1.555556
3,min,1,1,-1.000000,-1.000000,82.000000,1.000000,546.000000,9.000000,7.000000,73.000000,5.000000,62.000000,1.000000
4,stddev,None,None,1.415195,1.643825,1.222475,7.424621,4418.767228,144.035098,84.882124,180.210159,10.591795,185.761088,0.707107


In [12]:
snowpark_df.schema.fields

[StructField('ID', StringType(), nullable=True),
 StructField('PROJECT', StringType(), nullable=True),
 StructField('TEAMEXP', LongType(), nullable=True),
 StructField('MANAGEREXP', LongType(), nullable=True),
 StructField('YEAREND', LongType(), nullable=True),
 StructField('LENGTH', LongType(), nullable=True),
 StructField('EFFORT', LongType(), nullable=True),
 StructField('TRANSACTIONS', LongType(), nullable=True),
 StructField('ENTITIES', LongType(), nullable=True),
 StructField('POINTNONADJUST', LongType(), nullable=True),
 StructField('ADJUSTMENT', LongType(), nullable=True),
 StructField('POINTSADJUST', LongType(), nullable=True),
 StructField('LANGUAGE', LongType(), nullable=True)]

In [13]:
# # Imputing missing values in numeric columns
# numeric_types = [DecimalType(), LongType(), DoubleType(), FloatType(), IntegerType()]
# numeric_columns = [c.name for c in snowpark_df.schema.fields if type(c.datatype) in numeric_types]
# for column in numeric_columns:
#     snowpark_df = snowpark_df.fillna(snowpark_df[[fn.mean(column)]].collect()[0][0], column)

# # Check results
# print('Data after imputation:')
# display(snowpark_df.describe().sort('SUMMARY').toPandas())

### IMPUTE SOME NUMERIC X VARIABLES BY APPLYING AVERAGE directly on the base database table in snowflake

In [ ]:
spe_session.sql("update software_projects set avg_session_length = (select avg(avg_session_length) from software_projects where  avg_session_length is not null) where avg_session_length is null").collect()
spe_session.sql("update software_projects set TIME_ON_APP = (select avg(TIME_ON_APP) from software_projects where  TIME_ON_APP is not null) where TIME_ON_APP is null").collect()
spe_session.sql("update software_projects set TIME_ON_WEBSITE = (select avg(TIME_ON_WEBSITE) from software_projects where  TIME_ON_WEBSITE is not null) where TIME_ON_WEBSITE is null").collect()
spe_session.sql("update software_projects set MEMBERSHIP_LENGTH = (select avg(MEMBERSHIP_LENGTH) from software_projects where  MEMBERSHIP_LENGTH is not null) where MEMBERSHIP_LENGTH is null").collect()
spe_session.sql("update software_projects set YEARLY_SPENT = (select avg(YEARLY_SPENT) from software_projects where  YEARLY_SPENT is not null)where YEARLY_SPENT is null").collect()

### CHECK THE STATS AFTER IMPUTE

In [14]:
snowpark_df = spe_session.table('software_projects')

# Describing the data
print('Rows in dataset:', f"{snowpark_df.count():,}")
print('Data after imputation:')
display(snowpark_df.describe().sort('SUMMARY').toPandas())

Rows in dataset: 81
Data after imputation:


,SUMMARY,ID,PROJECT,TEAMEXP,MANAGEREXP,YEAREND,LENGTH,EFFORT,TRANSACTIONS,ENTITIES,POINTNONADJUST,ADJUSTMENT,POINTSADJUST,LANGUAGE
0,count,81,81,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
1,max,9,9,4.000000,7.000000,88.000000,39.000000,23940.000000,886.000000,387.000000,1127.000000,52.000000,1116.000000,3.000000
2,mean,None,None,2.185185,2.530864,85.740741,11.666667,5046.308642,182.123457,122.333333,304.456790,27.629630,289.234568,1.555556
3,min,1,1,-1.000000,-1.000000,82.000000,1.000000,546.000000,9.000000,7.000000,73.000000,5.000000,62.000000,1.000000
4,stddev,None,None,1.415195,1.643825,1.222475,7.424621,4418.767228,144.035098,84.882124,180.210159,10.591795,185.761088,0.707107


## HERE IS WHERE WE USE THE POWER OF SNOWFLAKE WHERE WE PUSH DOWN MODEL TRAINING TO SNOWFLAKE AND SNOWFLAKE CREDITS WILL BE CONSUMED. MAKE SURE YOUR WAREHOUSE IS RIGHTLY SIZED FOR THE VOLUME OF DATA YOU ARE TRAINING THE MODEL.

### DEFINE THE TRAINING MODEL AND FOR THIS USE CASE LET'S USE SKLEARN'S LINEAR REGRESSION MODEL

In [15]:
features = ['TEAMEXP','MANAGEREXP','YEAREND','LENGTH','TRANSACTIONS','ENTITIES','POINTNONADJUST','ADJUSTMENT','POINTSADJUST','LANGUAGE']

In [16]:
# This local Python-function will be registered as a Stored Procedure

def spec_train_lm_model(session: Session, training_table: str, sample_size_n: int, model_name: str,features:list) -> Variant:
    # Loading data into pandas dataframe
    local_training_data = session.table(training_table).sample(n=sample_size_n).toPandas()

    # Define features and label
    X = local_training_data[features]
    Y = local_training_data['EFFORT']

    # Splitting data into training and test
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=101)

    # Actual model training
    from sklearn.linear_model import LinearRegression
    lm = LinearRegression()
    lm.fit(X_train,y_train)
    
    # Getting model coefficients
    coeff_df = pd.DataFrame(lm.coef_,X.columns,columns=['Coefficient'])

    # Save model as file and upload to Snowflake stage
    from joblib import dump, load
    dump(lm, '/tmp/'+model_name)
    session.file.put('/tmp/'+model_name, '@knadadur_models', auto_compress=False)
    return coeff_df.to_dict()

In [26]:
# This local Python-function will be registered as a Stored Procedure

def spec_train_knn_model(session: Session, training_table: str, sample_size_n: int, model_name: str,features:list) -> Variant:
    # Loading data into pandas dataframe
    local_training_data = session.table(training_table).sample(n=sample_size_n).toPandas()

    # Define features and label
    X = local_training_data[features]
    Y = local_training_data['EFFORT']

    # Splitting data into training and test
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=30)

    # Actual model training
    from sklearn.neighbors import KNeighborsRegressor
    knn = KNeighborsRegressor(n_neighbors=3, weights='uniform')
    knn.fit(X_train,y_train)
    
    # Save model as file and upload to Snowflake stage
    from joblib import dump, load
    dump(knn, '/tmp/'+model_name)
    session.file.put('/tmp/'+model_name, '@knadadur_models', auto_compress=False)

    # Getting model score
    return knn.score(X_test, y_test)


### REGISTER THE "MODEL TRAINING" AS "STORED PROCEDURE" IN SNOWFLAKE USING THE DEFINED FUNCTION ABOVE

In [17]:
# Registering the function as a Stored Procedure
spec_linear_model_sproc = spe_session.sproc.register(func=spec_train_lm_model, # training function defined above
                                            name='spec_train_lm_model', # training model name to be registered in snowlake
                                            is_permanent=True, # permanent stored proc
                                            replace=True, # replace if existing already
                                            stage_location='@knadadur_models', # save the model in stage location
                                            packages=['snowflake-snowpark-python','scikit-learn','joblib']) 
                                                                        # import model libaries


In [27]:
# Registering the function as a Stored Procedure
spec_knn_sproc = spe_session.sproc.register(func=spec_train_knn_model, # training function defined above
                                            name='spec_train_knn_model', # training model name to be registered in snowlake
                                            is_permanent=True, # permanent stored proc
                                            replace=True, # replace if existing already
                                            stage_location='@knadadur_models', # save the model in stage location
                                            packages=['snowflake-snowpark-python','scikit-learn','joblib']) 
                                                                        # import model libaries


In [19]:
spe_session.sql("list @knadadur_models").collect()

[Row(name='knadadur_models/spec_train_lm_model.sav', size=1120, md5='812c306685523801fa990c6451313eaa', last_modified='Tue, 8 Nov 2022 06:06:25 GMT')]

### EXECUTE THE MODEL TRAINING FROM THE NOTEBOOK BUT THE EXECUTION HAPPENS IN SNOWFLAKE AND THE CREDITS WILL BE CONSUMED.

In [18]:
# Execute the Stored Procedure to train a model and display coefficients
spec_lm_coefficients = spec_linear_model_sproc('software_projects',100, 'spec_train_lm_model.sav',features)
#avoid passing the session parameter as remembers from the stored proc registered
#pass the train_set , data limit and the model file name to be saved
display(pd.DataFrame(eval(spec_lm_coefficients)))

,Coefficient
ADJUSTMENT,-115.172760
ENTITIES,-11.289223
LANGUAGE,-2045.495580
LENGTH,169.647656
MANAGEREXP,70.762307
POINTNONADJUST,-33.211573
POINTSADJUST,66.998785
TEAMEXP,-338.509993
TRANSACTIONS,-21.922350
YEAREND,326.371732


In [30]:
# Execute the Stored Procedure to train a model and display coefficients
spec_knn_score = spec_knn_sproc('software_projects',100, 'spec_train_knn_model.sav',features)
#avoid passing the session parameter as remembers from the stored proc registered
#pass the train_set , data limit and the model file name to be saved
spec_knn_score

'0.7450919920378207'

### CHECK IF THE MODEL FILE IS CREATED

In [ ]:
spe_session.sql("list @knadadur_models").collect()

## NOW THAT THE MODEL HAS BEEN TRAINED USING SNOWFLAKE STORED PROC AND THE MODEL FILE IS NOW SAVED IN THE STAGING ENVIRONMENT LET'S TRY TO INFER FROM THE MODEL TO PREDICT

### IMPORT THE LIBRARIES REQUIRED FOR INFERENCE

In [33]:
from snowflake.snowpark.functions import pandas_udf
from cachetools import cached

### DEFINE THE INFERENCE TO PREDICT FROM THE MODEL SAVED IN THE INTERNAL STAGE DIRECTORY IN SNOWFLAKE

In [40]:
@cached(cache={})
def load_model(model_path: str) -> object:
    from joblib import load
    model = load(model_path)
    return model
### define the datatypes for the number of input variables here in score_model function
def score_model(df: PandasDataFrame[int,int,int,int,int,int,int,int,int,int]) -> PandasSeries[float]:
    import os
    import sys
    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]
    model_name = 'spec_train_lm_model.sav'
    model = load_model(import_dir+model_name)
    return pd.Series(model.predict(df))

### DEFINE THE MODEL INFERENCE IN A USER DEFINED FUNCTION (UDF)

In [41]:
lm_model_udf = pandas_udf(func=score_model, 
                           name="spec_lm_prediction_vec", 
                           stage_location='@knadadur_models',
                           replace=True, 
                           is_permanent=True, 
                           imports=['@knadadur_models/spec_train_lm_model.sav'],
                           packages=['scikit-learn==1.0.2','pandas','joblib','cachetools'], 
                           max_batch_size=100, 
                           session=spe_session)

The version of package scikit-learn in the local environment is 1.1.2, which does not fit the criteria for the requirement scikit-learn==1.0.2. Your UDF might not work when the package version is different between the server and your local environment


In [42]:
@cached(cache={})
def load_model(model_path: str) -> object:
    from joblib import load
    model = load(model_path)
    return model
### define the datatypes for the number of input variables here in score_model function
def score_knn_model(df: PandasDataFrame[int,int,int,int,int,int,int,int,int,int]) -> PandasSeries[float]:
    import os
    import sys
    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]
    model_name = 'spec_train_knn_model.sav'
    model = load_model(import_dir+model_name)
    return pd.Series(model.predict(df))

In [43]:
knn_model_udf = pandas_udf(func=score_knn_model, 
                           name="spec_knn_prediction_vec", 
                           stage_location='@knadadur_models',
                           replace=True, 
                           is_permanent=True, 
                           imports=['@knadadur_models/spec_train_knn_model.sav'],
                           packages=['scikit-learn==1.0.2','pandas','joblib','cachetools'], 
                           max_batch_size=100, 
                           session=spe_session)

The version of package scikit-learn in the local environment is 1.1.2, which does not fit the criteria for the requirement scikit-learn==1.0.2. Your UDF might not work when the package version is different between the server and your local environment


### DEFINE THE FEATURES (X VARIABLES) TO INFER

### CALL THE INFERENCE LIKE HOW YOU WOULD ON A SIMPLE SQL STATEMENT USING THE vec_model_udf that was defined previously

In [44]:
output = snowpark_df.limit(10000).select('ID',
                                         'PROJECT',
                                      *features,
                                      fn.col('EFFORT').alias('ACTUAL_EFFORT'),
                                      lm_model_udf(*features).alias('lm_PREDICTED_EFFORT'), # calling the model running in Snowflake (user-defined function)
                                      knn_model_udf(*features).alias('knn_PREDICTED_EFFORT'), # calling the model running in Snowflake (user-defined function)
                                      (lm_model_udf(*features) - fn.col('ACTUAL_EFFORT')).alias('lm_DELTA'),                                      
                                      (knn_model_udf(*features) - fn.col('ACTUAL_EFFORT')).alias('knn_DELTA')
                                     ).toPandas()
output

,ID,PROJECT,TEAMEXP,MANAGEREXP,YEAREND,LENGTH,TRANSACTIONS,ENTITIES,POINTNONADJUST,ADJUSTMENT,POINTSADJUST,LANGUAGE,ACTUAL_EFFORT,LM_PREDICTED_EFFORT,KNN_PREDICTED_EFFORT,LM_DELTA,KNN_DELTA
0,1,1,1,4,85,12,253,52,305,34,302,1,5152,5962.453231,4911.666667,810.453231,-240.333333
1,2,2,0,0,86,4,197,124,321,33,315,1,5635,5856.703820,5196.333333,221.703820,-438.666667
2,3,3,4,4,85,1,40,60,100,18,83,1,805,1638.348630,1753.666667,833.348630,948.666667
3,4,4,0,0,86,5,200,119,319,30,303,1,3829,5624.986552,5196.333333,1795.986552,1367.333333
4,5,5,0,0,86,4,140,94,234,24,208,1,2149,4202.046207,2648.333333,2053.046207,499.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,77,77,4,4,85,12,229,169,398,39,414,3,1400,3900.553187,4365.666667,2500.553187,2965.666667
77,78,78,4,3,83,12,227,73,300,34,297,1,2800,4387.388842,4785.666667,1587.388842,1985.666667
78,79,79,4,4,82,24,395,193,588,40,617,1,9520,12313.531190,9753.333333,2793.531190,233.333333
79,80,80,4,3,86,12,469,176,645,43,697,3,5880,9112.480495,9753.333333,3232.480495,3873.333333


### TIME TO WRAP AND CLOSE THE SNOWFLAKE SESSION

In [ ]:
spe_session.close()
print('Finished!!!')